# Classical Machine Learning for Activity Recognition

This notebook trains and evaluates classical ML models:
- Random Forest
- XGBoost

We compare performance using cross-validation and comprehensive metrics.

In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Add src to path
sys.path.insert(0, str(Path("../src").resolve()))

from fittrack.data.ingestion import HARDataLoader, ACTIVITY_LABELS
from fittrack.data.preprocessing import (
    create_train_val_test_split,
    get_class_weights,
    get_sample_weights,
)
from fittrack.models.classical import (
    ModelConfig,
    train_random_forest,
    train_xgboost,
    train_with_tuning,
    cross_validate_model,
    get_top_features,
    ClassicalModelTrainer,
)
from fittrack.models.evaluation import (
    compute_metrics,
    plot_confusion_matrix,
    plot_roc_curves,
    plot_precision_recall_per_class,
    plot_model_comparison,
    print_classification_report,
    ModelEvaluator,
)

plt.style.use("seaborn-v0_8-whitegrid")
%matplotlib inline

FIGURES_DIR = Path("../docs/figures")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

## 1. Load and Preprocess Data

In [ ]:
# Load data
loader = HARDataLoader()
train_data, test_data = loader.load_all()

print(f"Training samples: {train_data.n_samples}")
print(f"Test samples: {test_data.n_samples}")
print(f"Features: {train_data.n_features}")
print(f"Classes: {train_data.n_classes}")

In [ ]:
# Create train/val/test split from training data
# (The original test set is held out for final evaluation)
split = create_train_val_test_split(
    train_data.X,
    train_data.y,
    val_size=0.15,
    test_size=0.0,  # We use the original test set
    normalize=True,
)

# Normalize the held-out test set using the same scaler
X_test = split.scaler.transform(test_data.X.values)
y_test = split.label_encoder.transform(test_data.y["activity"])

print(f"\nSplit sizes:")
print(f"  Train: {len(split.X_train)}")
print(f"  Validation: {len(split.X_val)}")
print(f"  Test (held out): {len(X_test)}")

In [ ]:
# Class names and weights
class_names = split.class_names
print(f"Classes: {class_names}")

# Compute class weights for handling imbalance
class_weights = get_class_weights(split.y_train)
print(f"\nClass weights: {class_weights}")

## 2. Train Random Forest Baseline

In [ ]:
# Train with default settings
rf_config = ModelConfig(n_estimators=100, max_depth=None, random_state=42)
rf_result = train_random_forest(
    split.X_train, split.y_train,
    split.X_val, split.y_val,
    config=rf_config,
)

print(f"\nRandom Forest Results:")
print(f"  Train Accuracy: {rf_result.train_score:.4f}")
print(f"  Validation Accuracy: {rf_result.val_score:.4f}")

In [ ]:
# Cross-validation
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced",
)

cv_scores = cross_validate_model(
    rf_model,
    split.X_train,
    split.y_train,
    cv=5,
)

print(f"\n5-Fold CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

## 3. Train XGBoost Baseline

In [ ]:
# Train XGBoost
xgb_config = ModelConfig(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
)
xgb_result = train_xgboost(
    split.X_train, split.y_train,
    split.X_val, split.y_val,
    config=xgb_config,
)

print(f"\nXGBoost Results:")
print(f"  Train Accuracy: {xgb_result.train_score:.4f}")
print(f"  Validation Accuracy: {xgb_result.val_score:.4f}")

## 4. Hyperparameter Tuning

In [ ]:
# Tune Random Forest (quick search for demo)
print("Tuning Random Forest...")
rf_tuned_result = train_with_tuning(
    "random_forest",
    split.X_train, split.y_train,
    split.X_val, split.y_val,
    search_type="random",
    cv=3,
    n_iter=20,
)

print(f"\nTuned Random Forest:")
print(f"  Best params: {rf_tuned_result.best_params}")
print(f"  Train Accuracy: {rf_tuned_result.train_score:.4f}")
print(f"  Validation Accuracy: {rf_tuned_result.val_score:.4f}")

In [ ]:
# Tune XGBoost
print("Tuning XGBoost...")
xgb_tuned_result = train_with_tuning(
    "xgboost",
    split.X_train, split.y_train,
    split.X_val, split.y_val,
    search_type="random",
    cv=3,
    n_iter=20,
)

print(f"\nTuned XGBoost:")
print(f"  Best params: {xgb_tuned_result.best_params}")
print(f"  Train Accuracy: {xgb_tuned_result.train_score:.4f}")
print(f"  Validation Accuracy: {xgb_tuned_result.val_score:.4f}")

## 5. Final Evaluation on Test Set

In [ ]:
# Initialize evaluator
evaluator = ModelEvaluator(class_names=class_names, figures_dir=FIGURES_DIR)

# Evaluate both models
rf_metrics = evaluator.evaluate(rf_tuned_result.model, X_test, y_test, "Random Forest")
xgb_metrics = evaluator.evaluate(xgb_tuned_result.model, X_test, y_test, "XGBoost")

print(evaluator.get_summary())

In [ ]:
# Classification report
print("\n" + "="*60)
print("RANDOM FOREST - Classification Report")
print("="*60)
y_pred_rf = rf_tuned_result.model.predict(X_test)
print_classification_report(y_test, y_pred_rf, class_names)

print("\n" + "="*60)
print("XGBOOST - Classification Report")
print("="*60)
y_pred_xgb = xgb_tuned_result.model.predict(X_test)
print_classification_report(y_test, y_pred_xgb, class_names)

## 6. Visualizations

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest
plot_confusion_matrix(
    rf_metrics.confusion_matrix,
    class_names,
    title="Random Forest - Confusion Matrix",
)
plt.savefig(FIGURES_DIR / "confusion_matrix_rf.png", dpi=150, bbox_inches="tight")
plt.show()

# XGBoost
plot_confusion_matrix(
    xgb_metrics.confusion_matrix,
    class_names,
    title="XGBoost - Confusion Matrix",
)
plt.savefig(FIGURES_DIR / "confusion_matrix_xgb.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# ROC Curves
y_proba_rf = rf_tuned_result.model.predict_proba(X_test)
y_proba_xgb = xgb_tuned_result.model.predict_proba(X_test)

plot_roc_curves(y_test, y_proba_rf, class_names, title="Random Forest - ROC Curves")
plt.savefig(FIGURES_DIR / "roc_curves_rf.png", dpi=150, bbox_inches="tight")
plt.show()

plot_roc_curves(y_test, y_proba_xgb, class_names, title="XGBoost - ROC Curves")
plt.savefig(FIGURES_DIR / "roc_curves_xgb.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Model comparison
plot_model_comparison(
    {"Random Forest": rf_metrics, "XGBoost": xgb_metrics},
    metric_names=["accuracy", "precision_macro", "recall_macro", "f1_macro"],
    title="Model Comparison on Test Set",
)
plt.savefig(FIGURES_DIR / "model_comparison_classical.png", dpi=150, bbox_inches="tight")
plt.show()

In [ ]:
# Per-class metrics
plot_precision_recall_per_class(
    rf_metrics,
    class_names,
    title="Random Forest - Per-Class Metrics",
)
plt.savefig(FIGURES_DIR / "per_class_metrics_rf.png", dpi=150, bbox_inches="tight")
plt.show()

## 7. Feature Importance Analysis

In [ ]:
# Top features from Random Forest
feature_names = train_data.X.columns.tolist()
top_features_rf = get_top_features(
    rf_tuned_result.feature_importances,
    feature_names,
    n_top=20,
)

print("Top 20 Features (Random Forest):")
for name, importance in top_features_rf:
    print(f"  {name}: {importance:.4f}")

In [ ]:
# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 8))

names, importances = zip(*top_features_rf)
y_pos = np.arange(len(names))

ax.barh(y_pos, importances, color="steelblue")
ax.set_yticks(y_pos)
ax.set_yticklabels(names)
ax.invert_yaxis()
ax.set_xlabel("Feature Importance")
ax.set_title("Top 20 Features - Random Forest")

plt.tight_layout()
plt.savefig(FIGURES_DIR / "feature_importance_rf.png", dpi=150, bbox_inches="tight")
plt.show()

## 8. Summary

### Key Findings:
- Both Random Forest and XGBoost achieve strong performance on the HAR dataset
- The models show high accuracy on static activities (sitting, standing, laying)
- Some confusion exists between similar activities (walking upstairs/downstairs)
- Gravity acceleration and body acceleration features are most predictive

### Next Steps:
- Try deep learning models (LSTM, CNN) for potentially better temporal pattern recognition
- Implement MLflow tracking for experiment management
- Deploy the best model as a REST API

In [ ]:
# Save best model info
print("\n" + "="*60)
print("BEST MODEL SUMMARY")
print("="*60)

if rf_metrics.f1_macro >= xgb_metrics.f1_macro:
    print(f"Best model: Random Forest")
    print(f"F1 Score (macro): {rf_metrics.f1_macro:.4f}")
    print(f"Accuracy: {rf_metrics.accuracy:.4f}")
    best_model = rf_tuned_result.model
else:
    print(f"Best model: XGBoost")
    print(f"F1 Score (macro): {xgb_metrics.f1_macro:.4f}")
    print(f"Accuracy: {xgb_metrics.accuracy:.4f}")
    best_model = xgb_tuned_result.model

print("="*60)